#  Задание 4

Для нормального распределения $X_i \sim N(\theta, \sigma ^2)$, если взять сумму независимых $X_i$, то доверительный интервал для $\theta$ будет:
$$P_{\theta}\left(\overline{x} - \frac{c_{1-\frac{\alpha}{2}} \sigma}{\sqrt{n}} < \theta < \overline{x} + \frac{c_{1-\frac{\alpha}{2}} \sigma}{\sqrt{n}} \right) = 1 - \alpha$$

Для **параметрического бутстрэпа** - оценим $\hat{\theta}$ и $\hat{\sigma ^2}$ по выборке и построим дополнительные выборки из распределения $X_i \sim N(\hat{\theta}, \hat{\sigma ^2})$

Для **не параметрического бутстрэпа** - из исходной выборки будем делать случайный выбор с возвратом.

## сделаем пункты 1-4 на 1 выборке

In [9]:
import numpy as np
from scipy import stats # чтобы считать квантили

In [19]:
def ci_theoretical(data, alpha=0.05):
    """дает кортеж из нижней и верхней границы доверительного интервала для среднего в нормальных x_i

    Args:
        data (array like): данные sample
        alpha (float, optional): уровень значимости. Defaults to 0.05.
    """
    std = np.std(data, ddof=1, dtype=np.float64)
    n = len(data)
    avg = np.mean(data)


    CI_Theoretical = (avg + stats.norm.ppf(alpha/2)*std /np.sqrt(n), avg + stats.norm.ppf(1-alpha/2)*std /np.sqrt(n))
    return CI_Theoretical


In [20]:

n = 50 # размер выборки 
alpha = 0.05 # параметр ошибки

np.random.seed(80085)
theta = np.random.uniform(10,20) # параметр тета
theor_sigma = np.random.uniform(1,2) # параметр тета
print((theta, theor_sigmasq))

(13.551121285103044, 1.0281182470223413)


In [21]:
sample1 = np.random.normal(theta,theor_sigma,size=n) 
sample1

array([13.101581  , 12.9629306 , 14.15844439, 13.1420225 , 13.21470771,
       12.92822876, 13.73439605, 13.62612039, 13.68009226, 12.11077405,
       13.64413202, 13.68589658, 11.13375744, 13.96078479, 11.5580977 ,
       14.58205901, 12.50967174, 14.29788902, 13.26076637, 13.87741364,
       13.49368701, 14.84815911, 15.17284706, 14.50394078, 12.42229787,
       14.07777694, 13.74842455, 14.05886051, 12.5529233 , 13.25924115,
       12.84479367, 13.04266122, 12.92148125, 12.97844854, 12.87426001,
       13.14309871, 12.68231808, 11.87790938, 13.13715342, 13.93000191,
       15.00731573, 11.89047398, 15.30802172, 12.37235166, 13.16092178,
       13.49004403, 14.48894427, 13.98446822, 14.10699862, 14.0301892 ])

In [22]:
ci_theoretical(sample1)

(13.158376544840669, 13.6648146417907)

## Сделаем это 10000 раз